<a href="https://colab.research.google.com/github/gongboogi/Python-Practice/blob/main/Pandas%EB%A1%9C%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%8B%A4%EB%A3%A8%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 판다스를 이용해 데이터 다루기

In [2]:
# 라이브러리 임포트
import numpy as np
import numpy.random as random
import scipy as sp
import pandas as pd
from pandas import Series, DataFrame

# 시각화 라이브러리
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
%matplotlib inline

# 소수점 세 번째 자리까지 표시
%precision 3

'%.3f'

# 판다스로 데이터를 다루는 기본적인 방법

## 1. 계층적 인덱스

데이터를 여러 개의 변수를 기준으로 집계할 때 계층적 인덱스가 편리하다. 다음 코드는 데이터 세트를 2층 인덱스 구조로 설정했다.

In [3]:
# 3열 3행 데이터를 생성하고 인덱스와 칼럼명 설정
hier_df = DataFrame(
    np.arange(9).reshape((3,3)),
    index = [
        ['a', 'a', 'b'],
        [1,2,2]
    ],

    columns = [
        ['Pusan', 'Seoul', 'Pusan'],
        ['Blue', 'Red', 'Red']
    ]
)

hier_df

Pusan Seoul Pusan
     Blue   Red   Red
a 1     0     1     2
  2     3     4     5
b 2     6     7     8

In [4]:
# index에 이름 붙이기: names
hier_df.index.names = ['key1', 'key2']

# 칼럼에 이름 붙이기: names
hier_df.columns.names = ['city', 'color']
hier_df

city      Pusan Seoul Pusan
color      Blue   Red   Red
key1 key2                  
a    1        0     1     2
     2        3     4     5
b    2        6     7     8

In [5]:
# 칼럼 범위 축소: city가 Pusan인 데이터만 보기
hier_df['Pusan']

color      Blue  Red
key1 key2           
a    1        0    2
     2        3    5
b    2        6    8

**인덱스 기준 집계**

In [6]:
# 계층별 요약통계량: 행 합계
hier_df.sum(level = 'key2', axis = 0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  


city,Pusan,Seoul,Pusan
color,Blue,Red,Red
key2,,,
1,0,1,2
2,9,11,13


In [7]:
# 열 합계: axis=1
hier_df.sum(level = 'color', axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  


color      Blue  Red
key1 key2           
a    1        0    3
     2        3    9
b    2        6   15

**인덱스 원소 삭제**

특정 인덱스를 삭제할 때는 drop 메서드를 사용한다.

In [8]:
#key1의 b 삭제
hier_df.drop(['b'])

city      Pusan Seoul Pusan
color      Blue   Red   Red
key1 key2                  
a    1        0     1     2
     2        3     4     5



---



##연습문제

1 . 다음 데이터에서 Daegu 열만 추출하세요.


In [9]:
hier_df1 = DataFrame(
  np.arange(12).reshape((3,4)),
  index = [['c', 'd', 'd'], [1,2,1]],
  columns = [
    ['Daegu', 'Daejeon', 'Gangeung', 'Daegu'],
    ['Yellow', 'Yellow', 'Red', 'Blue']
  ]
)

hier_df1.index.names = ['key1', 'key2']
hier_df1.columns.names = ['city', 'color']
hier_df1

city       Daegu Daejeon Gangeung Daegu
color     Yellow  Yellow      Red  Blue
key1 key2                              
c    1         0       1        2     3
d    2         4       5        6     7
     1         8       9       10    11

In [10]:
hier_df1['Daegu']

color      Yellow  Blue
key1 key2              
c    1          0     3
d    2          4     7
     1          8    11

2 . 1의 데이터에서 city별 평균값을 산출하세요.

In [11]:
hier_df1.mean(level = 'city', axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.median(level=1) should use df.groupby(level=1).median().
  """Entry point for launching an IPython kernel.


city       Daegu  Daejeon  Gangeung
key1 key2                          
c    1       1.5      1.0       2.0
d    2       5.5      5.0       6.0
     1       9.5      9.0      10.0

3 . 1의 데이터에서 key2별로 각 행의 합계를 산출하세요.

In [12]:
hier_df1.sum(level = 'key2')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  """Entry point for launching an IPython kernel.


city,Daegu,Daejeon,Gangeung,Daegu
color,Yellow,Yellow,Red,Blue
key2,,,,
1,8,10,12,14
2,4,5,6,7




---



## 2. 데이터 결합

In [13]:
# 데이터1 생성
data1 = {
    'id' : ['100','101','102', '103', '104', '106', '108', '110', '111', '113'],
    'city' : ['Soeul', 'Pusan', 'Daegu', 'Gangneung', 'Seoul', 'Seoul', 'Pusan', 'Daegu', 'Gangneung', 'Seoul'],
    'birth_year' : [1990, 1989, 1992, 1997, 1982, 1991, 1988, 1990, 1995, 1981],
    'name' : ['Junho', 'Heejin', 'Mijung','Minho', 'Steeve', 'Mina', 'Sumi', 'Minsu', 'Jinhee', 'Daeho']
}

df1 = DataFrame(data1)
df1

,id,city,birth_year,name
0,100,Soeul,1990,Junho
1,101,Pusan,1989,Heejin
2,102,Daegu,1992,Mijung
3,103,Gangneung,1997,Minho
4,104,Seoul,1982,Steeve
5,106,Seoul,1991,Mina
6,108,Pusan,1988,Sumi
7,110,Daegu,1990,Minsu
8,111,Gangneung,1995,Jinhee
9,113,Seoul,1981,Daeho


In [14]:
# 데이터2 생성
data2 = {
    'id' : ['100', '101', '102', '105', '107'],
    'math;' : [50, 43, 33, 76, 98],
    'english' : [90,30,20,50,30],
    'sex' : ['M', 'F', 'F', 'M', 'M'],
    'index_num' : [0,1,2,3,4]
}

df2 = DataFrame(data2)
df2

,id,math;,english,sex,index_num
0,100,50,90,M,0
1,101,43,30,F,1
2,102,33,20,F,2
3,105,76,50,M,3
4,107,98,30,M,4


**결합**

결합의 네 가지 패턴

1. **내부결합(INNSER JOIN)**: 양쪽 데이터에 공통 키 값이 있을 때 결합
2. **외부결합(FULL JOIN)**: 양쪽 모든 데이터를 결합
3. **왼쪽결합(LEFT JOIN)**: 왼쪽 데이터의 키 값을 기준으로 결합
4. **오른쪽결합(RIGHT JOIN)**: 오른쪽 데이터의 키 값을 기준으로 결합

주로 '내부결합'과 '왼쪽결합'을 많이 사용한다. <br>

**내부결합(INNER JOIN)**

merge 메서드는 내부결합이 기본값이다. id를 키 값으로 위 2개의 데이터를 내부 결합하면 아래와 같다. on 파라미터에서 키 값으로 사용할 변수(칼럼)를 지정한다.

In [15]:
# 데이터 결합(내부결합, 키는 자동으로 인식하지만 on으로 명시적으로 지정할 수 있다)
print('결합 테이블')
pd.merge(df1, df2, on = 'id')

결합 테이블


,id,city,birth_year,name,math;,english,sex,index_num
0,100,Soeul,1990,Junho,50,90,M,0
1,101,Pusan,1989,Heejin,43,30,F,1
2,102,Daegu,1992,Mijung,33,20,F,2


**외부결합(OUTER JOIN)**

양쪽 데이터를 모두 결합하는 전결합이다. 전결합 하기 위해 how 파라미터에 **outer**을 지정한다. 결합된 데이터의 특정 변수에 값이 없을 때 NaN으로 표시된다.

In [16]:
# 데이터 결합(전결합)
pd.merge(df1, df2, how = 'outer')

,id,city,birth_year,name,math;,english,sex,index_num
0,100,Soeul,1990.0,Junho,50.0,90.0,M,0.0
1,101,Pusan,1989.0,Heejin,43.0,30.0,F,1.0
2,102,Daegu,1992.0,Mijung,33.0,20.0,F,2.0
3,103,Gangneung,1997.0,Minho,NaN,NaN,NaN,NaN
4,104,Seoul,1982.0,Steeve,NaN,NaN,NaN,NaN
5,106,Seoul,1991.0,Mina,NaN,NaN,NaN,NaN
6,108,Pusan,1988.0,Sumi,NaN,NaN,NaN,NaN
7,110,Daegu,1990.0,Minsu,NaN,NaN,NaN,NaN
8,111,Gangneung,1995.0,Jinhee,NaN,NaN,NaN,NaN
9,113,Seoul,1981.0,Daeho,NaN,NaN,NaN,NaN


left_index 파라미터나 right_on 파라미터로 키를 인덱스로 지정해 결합할 수 있다. 다음 코드는 왼쪽 데이터의 인덱스와 오른쪽 데이터의 index_num 칼럼을 키로 지정한다.

In [17]:
# index를 이용한 결합
pd.merge(df1, df2, left_index = True, right_on = 'index_num')

,id_x,city,birth_year,name,id_y,math;,english,sex,index_num
0,100,Soeul,1990,Junho,100,50,90,M,0
1,101,Pusan,1989,Heejin,101,43,30,F,1
2,102,Daegu,1992,Mijung,102,33,20,F,2
3,103,Gangneung,1997,Minho,105,76,50,M,3
4,104,Seoul,1982,Steeve,107,98,30,M,4


**왼쪽결합(LEFT JOIN)**

왼쪽 결합은 how 파라미터를 left로 지정한다. 다음 예시는 왼쪽 데이터에 맞춰 DataFrame 객체 데이터를 결합한다. 왼쪽 데이터 기준변수(인수)에 대응하는 데이터가 오른쪽 데이터에(두 번째 인수) 없는 경우 해당 칼럼의 값은 NaN이 된다.

In [18]:
# 데이터 결합(left)
pd.merge(df1, df2, how = 'left')

,id,city,birth_year,name,math;,english,sex,index_num
0,100,Soeul,1990,Junho,50.0,90.0,M,0.0
1,101,Pusan,1989,Heejin,43.0,30.0,F,1.0
2,102,Daegu,1992,Mijung,33.0,20.0,F,2.0
3,103,Gangneung,1997,Minho,NaN,NaN,NaN,NaN
4,104,Seoul,1982,Steeve,NaN,NaN,NaN,NaN
5,106,Seoul,1991,Mina,NaN,NaN,NaN,NaN
6,108,Pusan,1988,Sumi,NaN,NaN,NaN,NaN
7,110,Daegu,1990,Minsu,NaN,NaN,NaN,NaN
8,111,Gangneung,1995,Jinhee,NaN,NaN,NaN,NaN
9,113,Seoul,1981,Daeho,NaN,NaN,NaN,NaN


**수직방향 결합**

앞에서는 특정 키를 기준으로 결합했지만,
 concat 메서드를 이용하면 데이터를 수직방향으로 쌓아 올릴 수 있다.

In [19]:
# 데이터3 생성
data3 = {
    'id' : ['117', '118', '119', '120', '125'],
    'city' : ['Ilsan', 'Gunpo', 'Seoul', 'Changwon', 'Jeju'],
    'birth_year' : [1990, 1989, 1992, 1997, 1982],
    'name' : ['Jinhee', 'Yeongho', 'Jongho', 'Yeonghee', 'Hyejin']
}

df3 = DataFrame(data3)
df3

,id,city,birth_year,name
0,117,Ilsan,1990,Jinhee
1,118,Gunpo,1989,Yeongho
2,119,Seoul,1992,Jongho
3,120,Changwon,1997,Yeonghee
4,125,Jeju,1982,Hyejin


In [20]:
# concat 수직방향 결합
concat_data = pd.concat([df1,df3])
concat_data

,id,city,birth_year,name
0,100,Soeul,1990,Junho
1,101,Pusan,1989,Heejin
2,102,Daegu,1992,Mijung
3,103,Gangneung,1997,Minho
4,104,Seoul,1982,Steeve
5,106,Seoul,1991,Mina
6,108,Pusan,1988,Sumi
7,110,Daegu,1990,Minsu
8,111,Gangneung,1995,Jinhee
9,113,Seoul,1981,Daeho




---



## 연습문제

4 . 아래 두 가지 데이터를 내부결합 하세요.

In [21]:
# 데이터4 생성
data4 = {
    'id' : ['0', '1', '2', '3', '4', '6', '8', '11', '12', '13'],
    'city' : ['Seoul', 'Pusan', 'Daegu', 'Gangneung', 'Seoul', 'Seoul',
              'Pusan', 'Daeug', 'Gangneung', 'Seoul'],
    'birth_year' : [1990, 1989, 1992, 1997, 1982, 1991, 1988, 1990, 1995, 1981],
    'name' : ['Junho', 'Heejin', 'Mijung', 'Minho', 'Steeve', 'Mina', 'Sumi', 'Minsu', 'Jinhee', 'Daeho']
}

df4 = DataFrame(data4)
df4

,id,city,birth_year,name
0,0,Seoul,1990,Junho
1,1,Pusan,1989,Heejin
2,2,Daegu,1992,Mijung
3,3,Gangneung,1997,Minho
4,4,Seoul,1982,Steeve
5,6,Seoul,1991,Mina
6,8,Pusan,1988,Sumi
7,11,Daeug,1990,Minsu
8,12,Gangneung,1995,Jinhee
9,13,Seoul,1981,Daeho


In [22]:
# 데이터5 생성
data5 = {
    'id' : ['0', '1', '3', '6', '8'],
    'math' : [20, 30, 50, 70, 90],
    'english' : [30,50,50,70,20],
    'sex' : ['M', 'F', 'F', 'M', 'M'],
    'index_num' : [0,1,2,3,4]
}
df5 = DataFrame(data5)
df5

,id,math,english,sex,index_num
0,0,20,30,M,0
1,1,30,50,F,1
2,3,50,50,F,2
3,6,70,70,M,3
4,8,90,20,M,4


In [23]:
# 내부결합
Q4_innerjoin = pd.merge(df4, df5)
Q4_innerjoin

,id,city,birth_year,name,math,english,sex,index_num
0,0,Seoul,1990,Junho,20,30,M,0
1,1,Pusan,1989,Heejin,30,50,F,1
2,3,Gangneung,1997,Minho,50,50,F,2
3,6,Seoul,1991,Mina,70,70,M,3
4,8,Pusan,1988,Sumi,90,20,M,4


5 . 문제 4의 데이터를 이용해 df4를 기준으로 df5 테이블을 전결합 하세요

In [24]:
Q5 = pd.merge(df4,df5, how = 'outer')
Q5

,id,city,birth_year,name,math,english,sex,index_num
0,0,Seoul,1990,Junho,20.0,30.0,M,0.0
1,1,Pusan,1989,Heejin,30.0,50.0,F,1.0
2,2,Daegu,1992,Mijung,NaN,NaN,NaN,NaN
3,3,Gangneung,1997,Minho,50.0,50.0,F,2.0
4,4,Seoul,1982,Steeve,NaN,NaN,NaN,NaN
5,6,Seoul,1991,Mina,70.0,70.0,M,3.0
6,8,Pusan,1988,Sumi,90.0,20.0,M,4.0
7,11,Daeug,1990,Minsu,NaN,NaN,NaN,NaN
8,12,Gangneung,1995,Jinhee,NaN,NaN,NaN,NaN
9,13,Seoul,1981,Daeho,NaN,NaN,NaN,NaN


6 . 문제4의 데이터를 이용해 df4를 기준으로 아래의 데이터를 수직방향으로 결합하세요.

In [25]:
# 데이터 생성
data6 = {
    'id' : ['70', '80', '90', '120', '150'],
    'city' : ['Ilsan', 'Gunpo', 'Seoul', 'Changwon', 'Jeju'],
    'birth_year' : [1980, 1999, 1995, 1994, 1994],
    'name' : ['Jinhee', 'Yeongho', 'Jongho', 'Yeonghee', 'Hyejin']
}

df6 = DataFrame(data6)
df6

,id,city,birth_year,name
0,70,Ilsan,1980,Jinhee
1,80,Gunpo,1999,Yeongho
2,90,Seoul,1995,Jongho
3,120,Changwon,1994,Yeonghee
4,150,Jeju,1994,Hyejin


In [26]:
# 수직방향 결합: concat
Q6 = pd.concat([df4, df6])
Q6

,id,city,birth_year,name
0,0,Seoul,1990,Junho
1,1,Pusan,1989,Heejin
2,2,Daegu,1992,Mijung
3,3,Gangneung,1997,Minho
4,4,Seoul,1982,Steeve
5,6,Seoul,1991,Mina
6,8,Pusan,1988,Sumi
7,11,Daeug,1990,Minsu
8,12,Gangneung,1995,Jinhee
9,13,Seoul,1981,Daeho


## 3. 데이터 조작과 변환

데이터를 조작 가공하고 변환하는 다양한 방법(피봇 처리, 중복 데이터 처리, 매핑, 구간화 등)

**데이터 피봇**

피봇(pivot)은 데이터의 행을 열로, 열을 행으로 바꾸는 작업이다.

In [27]:
# hier_df 생성
hier_df = DataFrame(
    np.arange(9).reshape((3,3)),
    index = [
        ['a', 'a', 'b'],
        [1,2,2]
    ],
    columns = [
        ['Pusan', 'Seoul', 'Pusan'],
        ['Blue', 'Red', 'Red']
    ]
)

hier_df

Pusan Seoul Pusan
     Blue   Red   Red
a 1     0     1     2
  2     3     4     5
b 2     6     7     8

In [28]:
# 피봇 기능으로 Blue, Red 열을 행으로 변경: stack
hier_df.stack()

Pusan  Seoul
a 1 Blue      0    NaN
    Red       2    1.0
  2 Blue      3    NaN
    Red       5    4.0
b 2 Blue      6    NaN
    Red       8    7.0

In [29]:
# unstack 메서드로 Blue, Red 행을 열로 변경
hier_df.stack().unstack()

Pusan     Seoul     
     Blue Red  Blue  Red
a 1     0   2   NaN  1.0
  2     3   5   NaN  4.0
b 2     6   8   NaN  7.0

**중복 데이터 제거**


In [30]:
# 중복 값이 있는 데이터
dupli_data = DataFrame({
  'col1' : [1,1,2,3,4,4,6,6],
  'col2' : ['a', 'b', 'b', 'b', 'c', 'c', 'b', 'b']
})
print('원본 데이터')
dupli_data

원본 데이터


,col1,col2
0,1,a
1,1,b
2,2,b
3,3,b
4,4,c
5,4,c
6,6,b
7,6,b


중복 데이터 여부를 확인하기 위해 duplicated 메서드를 사용한다. 모든 개별 행을 확인하고 중복이 있을 때는 True를 출력한다. 단, 중복 데이터에서 첫 번째 나오는 중복 데이터는 False, 두 번째 나올 때부터는 True를 출력.

In [31]:
# 중복 판정: duplicated
dupli_data.duplicated()

0    False
1    False
2    False
3    False
4    False
5     True
6    False
7     True
dtype: bool

In [32]:
# 중복 제거: drop_duplicates
dupli_data.drop_duplicates()

,col1,col2
0,1,a
1,1,b
2,2,b
3,3,b
4,4,c
6,6,b


**매핑**

매핑은 엑셀의 vlookup 함수와 같은 작업을 수행한다. 같은 키 값이 존재하는 두 개의 데이터에서 한쪽 데이터에서 다른 한쪽 데이터 키 값에 대응하는 데이터를 추출하는 기능이다.

In [33]:
# 참조 데이터
city_map = {
    'Seoul' : 'Sudo',
    'Gangneung' : 'Yeongdong',
    'Pusan' : 'Yeongnam',
    'Daegu' : 'Yeongnam'
}

city_map

{'Daegu': 'Yeongnam',
 'Gangneung': 'Yeongdong',
 'Pusan': 'Yeongnam',
 'Seoul': 'Sudo'}

다음은 df1에서 city 칼럼을 기준으로 위의 참조 데이터 city_map에 대응하는 지역명 데이터를 추출해 가장 오른쪽에 region칼럼으로 추가하는 작업이다.

In [34]:
df1

,id,city,birth_year,name
0,100,Soeul,1990,Junho
1,101,Pusan,1989,Heejin
2,102,Daegu,1992,Mijung
3,103,Gangneung,1997,Minho
4,104,Seoul,1982,Steeve
5,106,Seoul,1991,Mina
6,108,Pusan,1988,Sumi
7,110,Daegu,1990,Minsu
8,111,Gangneung,1995,Jinhee
9,113,Seoul,1981,Daeho


In [35]:
# 참조 데이터 결합
# 대응하는 데이터가 없으면 NaN이 된다

df1['region'] = df1['city'].map(city_map)
df1

,id,city,birth_year,name,region
0,100,Soeul,1990,Junho,NaN
1,101,Pusan,1989,Heejin,Yeongnam
2,102,Daegu,1992,Mijung,Yeongnam
3,103,Gangneung,1997,Minho,Yeongdong
4,104,Seoul,1982,Steeve,Sudo
5,106,Seoul,1991,Mina,Sudo
6,108,Pusan,1988,Sumi,Yeongnam
7,110,Daegu,1990,Minsu,Yeongnam
8,111,Gangneung,1995,Jinhee,Yeongdong
9,113,Seoul,1981,Daeho,Sudo


**익명함수와 map 함수 조합**

익명함수와 map 함수를 사용해 특정 칼럼 데이터에서 일부 데이터만 추출한다.

In [36]:
# birth_year에서 왼쪽부터 3개의 숫자, 문자를 추출
df1['up_two_num'] = df1['birth_year'].map(lambda x: str(x)[0:3])
df1

,id,city,birth_year,name,region,up_two_num
0,100,Soeul,1990,Junho,NaN,199
1,101,Pusan,1989,Heejin,Yeongnam,198
2,102,Daegu,1992,Mijung,Yeongnam,199
3,103,Gangneung,1997,Minho,Yeongdong,199
4,104,Seoul,1982,Steeve,Sudo,198
5,106,Seoul,1991,Mina,Sudo,199
6,108,Pusan,1988,Sumi,Yeongnam,198
7,110,Daegu,1990,Minsu,Yeongnam,199
8,111,Gangneung,1995,Jinhee,Yeongdong,199
9,113,Seoul,1981,Daeho,Sudo,198


**구간화**

데이터를 이산적인 구간으로 나누어 집계할 때 구간 분할 기능을 이용하면 편리하다. 판다스의 cut 함수를 이용해 구간을 만들 수 있다. 첫 번째 인수는 대상 데이터, 두 번째 인수는 구간을 나누느 경곗값을 지정한다.

In [37]:
df1.birth_year

0    1990
1    1989
2    1992
3    1997
4    1982
5    1991
6    1988
7    1990
8    1995
9    1981
Name: birth_year, dtype: int64

In [38]:
# 구간 분할 단위
birth_year_bins = [1980, 1985, 1990, 1995, 2000]

# 구간 분할 실행
birth_year_cut_data = pd.cut(df1.birth_year, birth_year_bins)
birth_year_cut_data

0    (1985, 1990]
1    (1985, 1990]
2    (1990, 1995]
3    (1995, 2000]
4    (1980, 1985]
5    (1990, 1995]
6    (1985, 1990]
7    (1985, 1990]
8    (1990, 1995]
9    (1980, 1985]
Name: birth_year, dtype: category
Categories (4, interval[int64, right]): [(1980, 1985] < (1985, 1990] < (1990, 1995] < (1995, 2000]]

구간으로 분할된 데이터의 구간별 개수를 집계할 때는 value_counts 함수를 이용한다.

In [39]:
# 집계 결과
pd.value_counts(birth_year_cut_data)

(1985, 1990]    4
(1990, 1995]    3
(1980, 1985]    2
(1995, 2000]    1
Name: birth_year, dtype: int64

labels 파라미터에서 각 구간의 이름을 지정할 수 있다.

In [40]:
# 이름 지정
group_names = ['early1980s', 'lave1980s', 'early1990s', 'late1990s']
birth_year_cut_data = pd.cut(df1.birth_year, birth_year_bins,
                             labels = group_names)
pd.value_counts(birth_year_cut_data)

lave1980s     4
early1990s    3
early1980s    2
late1990s     1
Name: birth_year, dtype: int64

위에서는 분할 구간을 리스트 데이터로 생성했지만 분할 구간 수를 지정할 때는 다음 코드처럼 설정할 수 있다. 


In [41]:
# 나눌 구간 수 지정 가능
pd.cut(df1.birth_year, 2)

0      (1989.0, 1997.0]
1    (1980.984, 1989.0]
2      (1989.0, 1997.0]
3      (1989.0, 1997.0]
4    (1980.984, 1989.0]
5      (1989.0, 1997.0]
6    (1980.984, 1989.0]
7      (1989.0, 1997.0]
8      (1989.0, 1997.0]
9    (1980.984, 1989.0]
Name: birth_year, dtype: category
Categories (2, interval[float64, right]): [(1980.984, 1989.0] < (1989.0, 1997.0]]

qcut 함수를 이용해 분위수를 기준으로 각 구간마다 거의 동일한 샘플 데이터 개수를 포함하도록 나눌 수 있다.

In [42]:
pd.qcut(df1.birth_year,2)

0    (1980.999, 1990.0]
1    (1980.999, 1990.0]
2      (1990.0, 1997.0]
3      (1990.0, 1997.0]
4    (1980.999, 1990.0]
5      (1990.0, 1997.0]
6    (1980.999, 1990.0]
7    (1980.999, 1990.0]
8      (1990.0, 1997.0]
9    (1980.999, 1990.0]
Name: birth_year, dtype: category
Categories (2, interval[float64, right]): [(1980.999, 1990.0] < (1990.0, 1997.0]]

In [43]:
pd.value_counts(pd.qcut(df1.birth_year,2))

(1980.999, 1990.0]    6
(1990.0, 1997.0]      4
Name: birth_year, dtype: int64

qcut 함수는 중앙값인 1990을 구간 분할 기준으로 사용한다. 

## 연습문제

7 . 수학 성적 데이터 student-mat.csv를 읽어 들여, 연령(age)에 2를 곱한 새로운 칼럼을 마지막 열에 추가하세요.

In [44]:
pwd

'/content'

In [45]:
mkdir python-practice

In [46]:
cd ./python-practice

/content/python-practice


In [47]:
import requests, zipfile
from io import StringIO
import io
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/00356/student.zip'
r = requests.get(url, stream = True)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

In [48]:
mat_data = pd.read_csv('student-mat.csv', sep=';')
mat_data.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10


In [49]:
mat_data['age2'] = mat_data['age'].map(lambda x: x*2)
mat_data.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3,age2
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,3,4,1,1,3,6,5,6,6,36
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,3,3,1,1,3,4,5,5,6,34
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,3,2,2,3,3,10,7,8,10,30
3,GP,F,15,U,GT3,T,4,2,health,services,...,2,2,1,1,5,2,15,14,15,30
4,GP,F,16,U,GT3,T,3,3,other,other,...,3,2,1,2,5,4,6,10,10,32


8 . 7과 동일한 데이터에서 absences 칼럼을 세 개의 구간으로 나누고 각 구간별 학생 수를 계산하세요. cut의 기본 설정은 오른쪽 구간에 포함되지 않는데(폐구간), 이번에는 cut 함수에서 right=False 옵션을 지정해 오른쪽 구간에 포함하세요(개구간).

In [50]:
# 구간 분할 간견
absences_bins = [0,1,5,100]

In [51]:
mat_cut_data = pd.cut(mat_data.absences, absences_bins, right = False)
pd.value_counts(mat_cut_data)

[5, 100)    151
[1, 5)      129
[0, 1)      115
Name: absences, dtype: int64

9 . 동일한 데이터에서 absences 칼럼을 qcut 함수로 세 개의 구간으로 분할하세요.

In [52]:
mat_qcut_data = pd.qcut(mat_data.absences, 3)
pd.value_counts(mat_qcut_data)

(-0.001, 2.0]    183
(6.0, 75.0]      115
(2.0, 6.0]        97
Name: absences, dtype: int64



---



## 4. 데이터 집계와 그룹 연산

특정 칼럼을 기준으로 집계하는 방법. 특정 변수를 기준으로 데이터를 집계할 때는 groupby 메서드를 사용한다.

In [53]:
# 데이터 준비, region 포함
df1

,id,city,birth_year,name,region,up_two_num
0,100,Soeul,1990,Junho,NaN,199
1,101,Pusan,1989,Heejin,Yeongnam,198
2,102,Daegu,1992,Mijung,Yeongnam,199
3,103,Gangneung,1997,Minho,Yeongdong,199
4,104,Seoul,1982,Steeve,Sudo,198
5,106,Seoul,1991,Mina,Sudo,199
6,108,Pusan,1988,Sumi,Yeongnam,198
7,110,Daegu,1990,Minsu,Yeongnam,199
8,111,Gangneung,1995,Jinhee,Yeongdong,199
9,113,Seoul,1981,Daeho,Sudo,198


In [54]:
# city별 데이터 개수 집계: groupby, size
df1.groupby('city').size()

city
Daegu        2
Gangneung    2
Pusan        2
Seoul        3
Soeul        1
dtype: int64

In [55]:
# city를 기준으로 birth_year 평균값
df1.groupby('city')['birth_year'].mean()

city
Daegu        1991.000000
Gangneung    1996.000000
Pusan        1988.500000
Seoul        1984.666667
Soeul        1990.000000
Name: birth_year, dtype: float64

In [56]:
# 복수 기준 설정 가능
df1.groupby(['region', 'city'])['birth_year'].mean()

region     city     
Sudo       Seoul        1984.666667
Yeongdong  Gangneung    1996.000000
Yeongnam   Daegu        1991.000000
           Pusan        1988.500000
Name: birth_year, dtype: float64

**as_index == False**

groupby 메서드에 **as_index == False** 파라미터를 설정하면 그룹별 출력 결과에 인덱스가 지정되지 않는다. 인덱스가 없는 형태의 데이터를 다뤄야 할 때 이용한다.

In [57]:
df1.groupby(['region', 'city'], as_index = False)['birth_year'].mean()

,region,city,birth_year
0,Sudo,Seoul,1984.666667
1,Yeongdong,Gangneung,1996.000000
2,Yeongnam,Daegu,1991.000000
3,Yeongnam,Pusan,1988.500000


**반복자(iterator)** 

groupby 메서드에는 값을 반복적으로 추출하는 반복자 기능이 있다.

In [58]:
for group, subdf in df1.groupby('region'):
  print('===========================')
  print('Region Name: {0}'.format(group))
  print(subdf)

Region Name: Sudo
    id   city  birth_year    name region up_two_num
4  104  Seoul        1982  Steeve   Sudo        198
5  106  Seoul        1991    Mina   Sudo        199
9  113  Seoul        1981   Daeho   Sudo        198
Region Name: Yeongdong
    id       city  birth_year    name     region up_two_num
3  103  Gangneung        1997   Minho  Yeongdong        199
8  111  Gangneung        1995  Jinhee  Yeongdong        199
Region Name: Yeongnam
    id   city  birth_year    name    region up_two_num
1  101  Pusan        1989  Heejin  Yeongnam        198
2  102  Daegu        1992  Mijung  Yeongnam        199
6  108  Pusan        1988    Sumi  Yeongnam        198
7  110  Daegu        1990   Minsu  Yeongnam        199


여러가지 계산을 한번에 모아 수행할 때는 agg 메서드가 편리하다. agg 메서드의 인수는 여러 개의 실행 함수를 리스트 데이터 형태로 전달한ㄷ. 

In [59]:
student_data_math = pd.read_csv('student-mat.csv', sep=';')

# 데이터 각 열에 복수의 함수 적용
functions = ['count', 'mean', 'max', 'min']
grouped_student_math_data1 = student_data_math.groupby(['sex', 'address'])
grouped_student_math_data1['age', 'G1'].agg(functions)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


age                       G1                   
            count       mean max min count       mean max min
sex address                                                  
F   R          44  16.977273  19  15    44  10.295455  19   6
    U         164  16.664634  20  15   164  10.707317  18   4
M   R          44  17.113636  21  15    44  10.659091  18   3
    U         143  16.517483  22  15   143  11.405594  19   5

# 연습문제

10 . 7의 student-mat.csv에서 학교(school) 변수를 기준으로 각 학교의 G1 평균 점수를 구하세요.

In [60]:
mat_data = pd.read_csv('student-mat.csv', sep = ';')
mat_data.groupby('school')['G1'].mean()

school
GP    10.939828
MS    10.673913
Name: G1, dtype: float64

11 . student-mat.csv에서 학교(school)와 성별(sex) 기준으로 G1, G2, G3 평균 점수를 구하세요.

In [61]:
mat_data.groupby(['school', 'sex'])['G1', 'G2', 'G3'].mean()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


G1         G2         G3
school sex                                 
GP     F    10.579235  10.398907   9.972678
       M    11.337349  11.204819  11.060241
MS     F    10.920000  10.320000   9.920000
       M    10.380952  10.047619   9.761905

12 . student-mat.csv에서 학교(school)의 성별(sex) 기준으로 G1, G2, G3의 최댓값, 최솟값을 한번에 계산하세요.

In [62]:
function12 = ['max', 'min']
groupped_mat_data = mat_data.groupby(['school', 'sex'])
groupped_mat_data['G1', 'G2', 'G3'].agg(function12)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


G1      G2      G3    
           max min max min max min
school sex                        
GP     F    18   4  18   0  19   0
       M    19   3  19   0  20   0
MS     F    19   6  18   5  19   0
       M    15   6  16   5  16   0



---



#**결측 데이터와 이상값 처리**

## 1. 결측 데이터 대처 방법

결측 데이터가 존재하는 원인은 매우 다양하다. 결측 데이터를 무시할지, 삭제할지 아니면 비슷한 값으로 대체할지 결정해야한다. 다음 예제 데이터는 값이  NaN(NA)로 된 부분이 결측 데이터다.

In [63]:
# 라이브러리 임포트
import numpy as np
from numpy import nan as NA
import pandas as pd

# 데이터 생성
df = pd.DataFrame(np.random.rand(10,4))

# NA로 변경
df.iloc[1,0] = NA
df.iloc[2:3, 2] = NA
df.iloc[5:, 3] = NA

In [64]:
df

,0,1,2,3
0,0.044360,0.290449,0.926907,0.896870
1,NaN,0.817609,0.700506,0.079638
2,0.228160,0.046894,NaN,0.737877
3,0.786574,0.514645,0.736845,0.731542
4,0.319247,0.363383,0.253514,0.124273
5,0.969957,0.968014,0.545873,NaN
6,0.127669,0.090856,0.043839,NaN
7,0.895502,0.641191,0.688253,NaN
8,0.893834,0.066702,0.854292,NaN
9,0.363696,0.157020,0.124544,NaN


**리스트와이즈삭제**

NaN이 있는 행을 모두 삭제하려면 dropna 메서드를 사용한다. 이 방법을 리스트와이즈(listwise) 삭제라고 한다.

In [65]:
df.dropna()

,0,1,2,3
0,0.044360,0.290449,0.926907,0.896870
3,0.786574,0.514645,0.736845,0.731542
4,0.319247,0.363383,0.253514,0.124273


**페어와이즈 삭제**

위의 결과에서 리스트와이즈 삭제로 원래 10행이었던 데이터가 3행만 남았다. 이렇게 데이터가 극단적으로 작아지면 데이터를 사용할 수 없는 상황이 발생할 수도 있다.

 반면 결측 데이터가 있는 열의 데이터는 무시하고 사용 가능한 데이터만 활용하는 방법이 있다. 이를 페어와이즈(pairwise)삭제라고 한다. 페어와이즈 삭제를 위해서 필요한 열을 추출하고 dropna 메서드를 적용한다.

In [66]:
df[[0,1]].dropna()

,0,1
0,0.044360,0.290449
2,0.228160,0.046894
3,0.786574,0.514645
4,0.319247,0.363383
5,0.969957,0.968014
6,0.127669,0.090856
7,0.895502,0.641191
8,0.893834,0.066702
9,0.363696,0.157020


**filna로 채우기**

fillna(값)으로 NaN을 괄호 안의 값으로 대체할 수 있다.

In [67]:
df.fillna(0)

,0,1,2,3
0,0.044360,0.290449,0.926907,0.896870
1,0.000000,0.817609,0.700506,0.079638
2,0.228160,0.046894,0.000000,0.737877
3,0.786574,0.514645,0.736845,0.731542
4,0.319247,0.363383,0.253514,0.124273
5,0.969957,0.968014,0.545873,0.000000
6,0.127669,0.090856,0.043839,0.000000
7,0.895502,0.641191,0.688253,0.000000
8,0.893834,0.066702,0.854292,0.000000
9,0.363696,0.157020,0.124544,0.000000


**이웃하는 값으로 대체하기**

ffill 메서드를 이용해 이웃하는 값(직전 행의 값)으로 대체할 수 있다. 이 방법은 금융시계열 데이터에서 활용할 수 있다.

In [68]:
df.fillna(method = 'ffill')

,0,1,2,3
0,0.044360,0.290449,0.926907,0.896870
1,0.044360,0.817609,0.700506,0.079638
2,0.228160,0.046894,0.700506,0.737877
3,0.786574,0.514645,0.736845,0.731542
4,0.319247,0.363383,0.253514,0.124273
5,0.969957,0.968014,0.545873,0.124273
6,0.127669,0.090856,0.043839,0.124273
7,0.895502,0.641191,0.688253,0.124273
8,0.893834,0.066702,0.854292,0.124273
9,0.363696,0.157020,0.124544,0.124273


**평군값으로 대체하기**

mean 메서드를 이용한다. 단, 시계열 데이터에서 이 방법을 적용하면 미래 정보를 포함시키게 되므로 주의해야한다.

In [69]:
# 각 칼럼의 평균값(확인용)
df.mean()

0    0.514333
1    0.395676
2    0.541619
3    0.514040
dtype: float64

In [70]:
df.fillna(df.mean())

,0,1,2,3
0,0.044360,0.290449,0.926907,0.896870
1,0.514333,0.817609,0.700506,0.079638
2,0.228160,0.046894,0.541619,0.737877
3,0.786574,0.514645,0.736845,0.731542
4,0.319247,0.363383,0.253514,0.124273
5,0.969957,0.968014,0.545873,0.514040
6,0.127669,0.090856,0.043839,0.514040
7,0.895502,0.641191,0.688253,0.514040
8,0.893834,0.066702,0.854292,0.514040
9,0.363696,0.157020,0.124544,0.514040


In [71]:
?df.fillna

# 연습문제

13 . 아래 데이터에서 1열이라도 NaN이 있는 경우 삭제하고 그 결과를 출력하세요.

In [72]:
# 라이브러리 임포트
import numpy as np
from numpy import nan as NA
import pandas as pd

# 데이터 생성
df2 = pd.DataFrame(np.random.rand(15,6))

# NA로 설정
df2.iloc[2,0] = NA
df2.iloc[5:8, 2] = NA
df2.iloc[7:9, 3] = NA
df2.iloc[10,5]=NA

df2

,0,1,2,3,4,5
0,0.837571,0.241419,0.881751,0.158788,0.010785,0.662312
1,0.032209,0.881282,0.207505,0.519077,0.729334,0.572917
2,NaN,0.946107,0.277004,0.992104,0.589011,0.113750
3,0.163351,0.140040,0.515207,0.497796,0.145861,0.791216
4,0.163357,0.635868,0.715071,0.729489,0.761094,0.400969
5,0.515991,0.434398,NaN,0.716949,0.087755,0.820480
6,0.177920,0.290569,NaN,0.839664,0.748260,0.216432
7,0.692497,0.716224,NaN,NaN,0.474980,0.374206
8,0.835763,0.767665,0.723529,NaN,0.950992,0.357478
9,0.627645,0.947908,0.496849,0.009546,0.116110,0.798840


In [73]:
df2.dropna()

,0,1,2,3,4,5
0,0.837571,0.241419,0.881751,0.158788,0.010785,0.662312
1,0.032209,0.881282,0.207505,0.519077,0.729334,0.572917
3,0.163351,0.140040,0.515207,0.497796,0.145861,0.791216
4,0.163357,0.635868,0.715071,0.729489,0.761094,0.400969
9,0.627645,0.947908,0.496849,0.009546,0.116110,0.798840
11,0.033498,0.571743,0.719720,0.427292,0.753360,0.670904
12,0.020985,0.436003,0.523147,0.700990,0.687713,0.366437
13,0.514104,0.864083,0.166260,0.248406,0.184450,0.782046
14,0.524207,0.671166,0.197116,0.869371,0.890333,0.714461


14 . 13에서 생성한 데이터에서 NaN을 0으로 대체하세요.

In [74]:
df2.fillna(0)

,0,1,2,3,4,5
0,0.837571,0.241419,0.881751,0.158788,0.010785,0.662312
1,0.032209,0.881282,0.207505,0.519077,0.729334,0.572917
2,0.000000,0.946107,0.277004,0.992104,0.589011,0.113750
3,0.163351,0.140040,0.515207,0.497796,0.145861,0.791216
4,0.163357,0.635868,0.715071,0.729489,0.761094,0.400969
5,0.515991,0.434398,0.000000,0.716949,0.087755,0.820480
6,0.177920,0.290569,0.000000,0.839664,0.748260,0.216432
7,0.692497,0.716224,0.000000,0.000000,0.474980,0.374206
8,0.835763,0.767665,0.723529,0.000000,0.950992,0.357478
9,0.627645,0.947908,0.496849,0.009546,0.116110,0.798840


15 . 13에서 생성한 데이터에서 NaN을 각 열의 평균값으로 대체하세요.

In [75]:
df2.fillna(df.mean())

,0,1,2,3,4,5
0,0.837571,0.241419,0.881751,0.158788,0.010785,0.662312
1,0.032209,0.881282,0.207505,0.519077,0.729334,0.572917
2,0.514333,0.946107,0.277004,0.992104,0.589011,0.113750
3,0.163351,0.140040,0.515207,0.497796,0.145861,0.791216
4,0.163357,0.635868,0.715071,0.729489,0.761094,0.400969
5,0.515991,0.434398,0.541619,0.716949,0.087755,0.820480
6,0.177920,0.290569,0.541619,0.839664,0.748260,0.216432
7,0.692497,0.716224,0.541619,0.514040,0.474980,0.374206
8,0.835763,0.767665,0.723529,0.514040,0.950992,0.357478
9,0.627645,0.947908,0.496849,0.009546,0.116110,0.798840




---



# **시계열 데이터 분석 방법 기초**


판다스를 이용하여 시계열 데이터를 다루는 방법.

In [76]:
import pandas_datareader.data as pdr

## 1. 시계열 데이터 조작과 변환

예제 데이터에서 2001/1/2부터 2016/12/30까지의 엔화 활율 데이터 사용. 일별 데이터이며 공휴일로 인한 결측 데이터 존재

In [77]:
start_date = '2001/1/2'
end_date = '2016/12/30'

fx_jpusdata = pdr.DataReader('DEXJPUS', 'fred', start_date, end_date)

In [78]:
fx_jpusdata.head()

,DEXJPUS
DATE,
2001-01-02,114.73
2001-01-03,114.26
2001-01-04,115.47
2001-01-05,116.19
2001-01-08,115.97


**특정 연월 데이터 참조**


In [79]:
fx_jpusdata['2016-04']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  """Entry point for launching an IPython kernel.


,DEXJPUS
DATE,
2016-04-01,112.06
2016-04-04,111.18
2016-04-05,110.26
2016-04-06,109.63
2016-04-07,107.98
2016-04-08,108.36
2016-04-11,107.96
2016-04-12,108.54
2016-04-13,109.21


In [80]:
# 월말 환율만 추출
fx_jpusdata.resample('M').last().head()

,DEXJPUS
DATE,
2001-01-31,116.39
2001-02-28,117.28
2001-03-31,125.54
2001-04-30,123.57
2001-05-31,118.88


일별로 데이터를 추출하려면 D, 연도별로 추출하려면 Y를 인수로 지정한다.

이와 같이 어떤 빈도 데이터를 별개의 빈도 데이터 기준으로 재추출하는 것을 리샘플링이라고 한다.

**결측 데이터가 있을 경우 처리 방법**

시계열 데이터에 결측 데이터가 있는 경우

In [81]:
fx_jpusdata.resample('D').last().head()

,DEXJPUS
DATE,
2001-01-02,114.73
2001-01-03,114.26
2001-01-04,115.47
2001-01-05,116.19
2001-01-06,NaN


In [82]:
# 직전 날의 값으로 대체
fx_jpusdata.resample('D').ffill().head()

,DEXJPUS
DATE,
2001-01-02,114.73
2001-01-03,114.26
2001-01-04,115.47
2001-01-05,116.19
2001-01-06,116.19


**데이터를 이동시켜 비율 계산**

전날의 환율과 비교하는 경우. shift 메서드로 인덱스는 고정하고 데이터는 이동할 수 있다.

In [83]:
fx_jpusdata.shift(1).head()

,DEXJPUS
DATE,
2001-01-02,NaN
2001-01-03,114.73
2001-01-04,114.26
2001-01-05,115.47
2001-01-08,116.19


In [84]:
fx_jpusdata_ratio = fx_jpusdata / fx_jpusdata.shift(1)
fx_jpusdata_ratio.head()

,DEXJPUS
DATE,
2001-01-02,NaN
2001-01-03,0.995903
2001-01-04,1.010590
2001-01-05,1.006235
2001-01-08,0.998107


위와 같이 데이터를 이동시켜 전날과 당일의 비율을 비교할 수 있다. 

diff와 pct_change를 이용해 차이와 비율을 구할 수 있다.

# 연습문제

16 . fx_jpusdata을 이용해 연도별 평균값 추세 데이터를 만드세요.

In [89]:
fx_jpusdata.resample('Y').mean().head()

,DEXJPUS
DATE,
2001-12-31,121.568040
2002-12-31,125.220438
2003-12-31,115.938685
2004-12-31,108.150830
2005-12-31,110.106932


## 2 . 이동평균

이동 평균은 시계열 데이터를 분석할 때 자주 활용된다.

앞에서 사용한 데이터로 3일 이동 평균선을 만들 수 있다.

In [85]:
fx_jpusdata.head()

,DEXJPUS
DATE,
2001-01-02,114.73
2001-01-03,114.26
2001-01-04,115.47
2001-01-05,116.19
2001-01-08,115.97


판다스의 rolling메서드로 쉽계 계산할 수 있다. 다음 코드는 3일 이동평균을 계산한 결과다.

In [86]:
fx_jpusdata.rolling(3).mean().head()

,DEXJPUS
DATE,
2001-01-02,NaN
2001-01-03,NaN
2001-01-04,114.820000
2001-01-05,115.306667
2001-01-08,115.876667


이동 평균이 아니라 표준편차 추세를 계산하려면 mean 대신 std 메서드를 사용한다. 

In [87]:
fx_jpusdata.rolling(3).std().head()

,DEXJPUS
DATE,
2001-01-02,NaN
2001-01-03,NaN
2001-01-04,0.610000
2001-01-05,0.975312
2001-01-08,0.368963


# 연습문제


17 . 16의 fx_jpusdata 데이터를 이용해 20일 이동평균 데이터를 만드세요. 단, NaN은 삭제하고 원래 존재하지 않는 결측 데이터 값은 대체할 필요는 없습니다.

In [90]:
fx_jpusdata_rolling20 = fx_jpusdata.rolling(20).mean().dropna()
fx_jpusdata_rolling20.head()

,DEXJPUS
DATE,
2001-02-12,116.6910
2001-02-13,116.6920
2001-02-14,116.6070
2001-02-15,116.5015
2001-02-16,116.4130


# 종합문제

수학 성적 데이터(student-mat.csv)를 이용하세요.

1. 연령(age) x 성별(sex) 기준으로 G1 평균을 계산하고 세로축이 연령(age), 가로축이 성별(sex)인 표를 만드세요.

In [96]:
student_data_math = pd.read_csv('student-mat.csv', sep = ';')

student_data_math.groupby(['age', 'sex'])['G1'].mean().unstack()

sex,F,M
age,,
15,10.052632,12.250000
16,10.203704,11.740000
17,11.103448,10.600000
18,10.883721,10.538462
19,10.642857,9.700000
20,15.000000,13.000000
21,NaN,10.000000
22,NaN,6.000000


2. 1번 문제에서 만든 표에서 NaN인 행 데이터를 모두 제거한 결과를 출력하세요.

In [97]:
student_data_math.groupby(['age', 'sex'])['G1'].mean().unstack().dropna()

sex,F,M
age,,
15,10.052632,12.250000
16,10.203704,11.740000
17,11.103448,10.600000
18,10.883721,10.538462
19,10.642857,9.700000
20,15.000000,13.000000
